<a href="https://colab.research.google.com/github/cleberjamaral/autoOrgDesignProject/blob/master/GoOrg4Prod_structure_choosing_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import altair as alt # for charts
from scipy import stats, spatial

In [2]:
bound_df = pd.read_csv("output/statistics/Feed_production_line_binding.csv","\t")
#skills = ["db_access","move","lift","pnp"]
skills = ["move","lift","pnp"]

In [3]:
bound_df.head()

,id,nPosit,%Idle+,%Effic,%Geral,%Speci,%Tall,%Flat,%Feasi,Levels,...,rWL,bDL,rDL,%WL+,%DL+,pTree,bgTree,skTree,agents,matche
0,1,1,46%,54%,100%,0%,0%,100%,0%,1,...,13.0,16.0,0.0,100%,0%,"[G{[GetBox, MoveBox, PlaceBox]}]","[PlaceBox, GetBox, MoveBox]","[[pnp, move, lift]]","[pp, bt, ie]",NaN
1,2,2,73%,27%,25%,75%,0%,100%,50%,1,...,13.0,16.0,8.0,100%,50%,"[G{[GetBox, MoveBox]}, G{[PlaceBox]}]","[PlaceBox, GetBox, MoveBox]","[[move, lift], [pnp]]","[pp, bt, ie]",p1=pp
2,3,2,73%,27%,25%,75%,0%,100%,50%,1,...,13.0,16.0,16.0,100%,100%,"[G{[GetBox, PlaceBox]}, G{[MoveBox]}]","[PlaceBox, GetBox, MoveBox]","[[move], [pnp, lift]]","[pp, bt, ie]",p1=bt
3,4,2,73%,27%,25%,75%,0%,100%,50%,1,...,13.0,16.0,8.0,100%,50%,"[G{[GetBox]}, G{[MoveBox, PlaceBox]}]","[PlaceBox, GetBox, MoveBox]","[[lift], [pnp, move]]","[pp, bt, ie]",p1=ie
4,5,2,73%,27%,25%,75%,50%,50%,50%,2,...,13.0,16.0,8.0,100%,50%,"[G{[GetBox, MoveBox]}, G{[PlaceBox]}^G{[GetBox...","[PlaceBox, GetBox, MoveBox]","[[move, lift], [pnp]]","[pp, bt, ie]",p1=pp


In [4]:
# Plotting configurations
CHART_WIDTH = 600
CHART_HEIGHT = 400

In [5]:
#bound_df.dtypes

In [6]:
# Generality similarity
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "similarity"])
dfsimilarity.columns = dfsimilarity.columns.map(str)
new_i = -1

# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        # df.values[row, column] = value
        if (new_i != i):
            new_i = i
            #print("Processing line ",i)
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i], 
            bound_df["id"][j], 
            100-abs(round((float(bound_df['%Geral'][i].rstrip('%'))-float(bound_df['%Geral'][j].rstrip('%')))))
         )

In [7]:
dfsimilarity

,id_a,id_b,similarity
0,1,1,100
1,1,2,25
2,1,3,25
3,1,4,25
4,1,5,25
...,...,...,...
671,26,22,100
672,26,23,100
673,26,24,100
674,26,25,100


In [8]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Similarity': dfsimilarity["similarity"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Similarity:Q'
).properties(
    title=["Generality Similarity Comparison"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Similarity:Q',
    color=alt.condition(
        alt.datum.Similarity < 70,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [9]:
# Efficiency similarity
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "similarity"])
dfsimilarity.columns = dfsimilarity.columns.map(str)
new_i = -1

# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        # df.values[row, column] = value
        if (new_i != i):
            new_i = i
            #print("Processing line ",i)
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i], 
            bound_df["id"][j], 
            100-abs(round((float(bound_df['%Effic'][i].rstrip('%'))-float(bound_df['%Effic'][j].rstrip('%')))))
         )

In [10]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Similarity': dfsimilarity["similarity"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Similarity:Q'
).properties(
    title=["Efficiency Similarity Comparison"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Similarity:Q',
    color=alt.condition(
        alt.datum.Similarity < 80,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [11]:
# Flatness similarity
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "similarity"])
dfsimilarity.columns = dfsimilarity.columns.map(str)
new_i = -1

# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i], 
            bound_df["id"][j], 
            100-abs(round((float(bound_df['%Flat'][i].rstrip('%'))-float(bound_df['%Flat'][j].rstrip('%')))))
         )

In [12]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Similarity': dfsimilarity["similarity"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Similarity:Q'
).properties(
    title=["Height Similarity Comparison"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Similarity:Q',
    color=alt.condition(
        alt.datum.Similarity < 70,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [13]:
skills = ["move","lift","pnp"]

In [14]:
def powerset(s):
    P = []
    x = len(s)
    for i in range(1 << x):
        P.append([s[j] for j in range(x) if (i & (1 << j))])
    return P

In [21]:
def get_organisations(array_str):
    organisations = []
    for i in range(0,len(array_str)):
        positions = []
        positions_str = array_str[i].split("G{")
        for j in range(0,len(positions_str)):
            position_str = positions_str[j].replace("[","").replace("]","").replace("}","").replace("^","").split(",")
            #print("p",j,position_str)
            position = []
            for k in range(0,len(position_str)):
                goal = position_str[k].strip()
                skill = goal.replace("MoveBox","move").replace("GetBox","lift").replace("PlaceBox","pnp")
                #print("g",k,goal)
                if len(goal)>0:
                    position.append(skill)
                #print(position)
            if len(position)>0:
                positions.append(position)
        #print(positions)
        organisations.append(positions)
    return organisations

In [16]:
# For each organisation, check which kinds of agent it has
all_agent_kinds = powerset(skills)
all_kinds = []
orgs = get_organisations(bound_df["pTree"])

def check_all_kinds_of_agents_in_orgs():
    for o in range(0,len(orgs)):
        kinds = []
        for k in range(0,len(all_agent_kinds)):
            if all_agent_kinds[k] in orgs[o]:
                kinds.append(k)
                #print(all_agent_kinds[k],">>",orgs[o],"::",kinds)
        all_kinds.append(kinds)
        
check_all_kinds_of_agents_in_orgs()

In [17]:
# Flatness similarity
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "similarity"])
dfsimilarity.columns = dfsimilarity.columns.map(str)

# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i], 
            bound_df["id"][j],
            round(100*(max(all_kinds)[0]-len(list(set(all_kinds[i]) - set(all_kinds[j])) + list(set(all_kinds[j]) - set(all_kinds[i]))))/max(all_kinds)[0])
         )

In [18]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Difference': dfsimilarity["similarity"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Difference:Q'
).properties(
    title=["Kind of Agents Similarity Comparison"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Difference:Q',
    color=alt.condition(
        alt.datum.Difference < 70,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [19]:
# Sum of similarities
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "similarity"])
dfsimilarity.columns = dfsimilarity.columns.map(str)
new_i = -1

# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i],
            bound_df["id"][j],
            round((
            100-abs(round((float(bound_df['%Geral'][i].rstrip('%'))-float(bound_df['%Geral'][j].rstrip('%')))))+
            100-abs(round((float(bound_df['%Effic'][i].rstrip('%'))-float(bound_df['%Effic'][j].rstrip('%')))))+
            100-abs(round((float(bound_df['%Flat'][i].rstrip('%'))-float(bound_df['%Flat'][j].rstrip('%')))))+
            round(100*(max(all_kinds)[0]-len(list(set(all_kinds[i]) - set(all_kinds[j])) + list(set(all_kinds[j]) - set(all_kinds[i]))))/max(all_kinds)[0])
            )/4)
         )

In [20]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Similarity': dfsimilarity["similarity"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Similarity:Q'
).properties(
    title=["Sum of Similarities Comparison"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Similarity:Q',
    color=alt.condition(
        alt.datum.Similarity < 70,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)